In [ ]:
# Install necessary packages (uncomment if needed)
# !pip install huggingface_hub
# !pip install transformers torch pandas scikit-learn

import transformers
import pandas as pd
import torch
import re

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

from huggingface_hub import login
from google.colab import userdata
hf_token = userdata.get('Llam3.1instruct')
login(token=hf_token)

# Set the Meta Llama model checkpoint
model_id = 'meta-llama/Llama-3.1-8B-Instruct'

# Create a text-generation pipeline with half-precision on GPU using Meta Llama
text_gen_pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# ---------------------------
# Agent 1: User Profiler
# ---------------------------
class UserProfiler:
    def __init__(self):
        # Define simple keyword mappings for movies to genres
        self.movie_genre_keywords = {
            "sci-fi": ["matrix", "inception", "avatar", "star wars", "interstellar"],
            "romance": ["titanic", "notebook", "romance", "love"],
            "action": ["avengers", "fast", "furious", "mission", "die hard"],
            "drama": ["drama", "joker", "fight club", "schindler"],
            "comedy": ["comedy", "funny", "hilarious", "dumb", "stupid"],
            "thriller": ["thriller", "mystery", "suspense", "intense"]
        }
        # Keywords in comments that signal a preference for certain genres
        self.comment_keywords = {
            "sci-fi": ["space", "future", "robot", "alien", "mind-bending"],
            "romance": ["heartwarming", "loving", "passionate", "emotional"],
            "action": ["exciting", "explosive", "adrenaline", "edge"],
            "drama": ["emotional", "deep", "serious"],
            "comedy": ["funny", "laugh", "amusing"],
            "thriller": ["suspense", "nail-biting", "intense"]
        }

    def profile_user(self, watch_history, comments):
        """
        Analyze the user's watch history and comments to determine preferred genres.
        """
        genre_scores = {}
        # Process watch history (list of movie titles)
        for movie in watch_history:
            movie_lower = movie.lower()
            for genre, keywords in self.movie_genre_keywords.items():
                for keyword in keywords:
                    if keyword in movie_lower:
                        genre_scores[genre] = genre_scores.get(genre, 0) + 1

        # Process comments (list of comments)
        for comment in comments:
            comment_lower = comment.lower()
            for genre, keywords in self.comment_keywords.items():
                for keyword in keywords:
                    if keyword in comment_lower:
                        genre_scores[genre] = genre_scores.get(genre, 0) + 1

        # Sort genres by score and return the list of genres with nonzero scores
        preferred_genres = sorted(genre_scores, key=genre_scores.get, reverse=True)
        return preferred_genres

# ---------------------------
# Agent 2: Preference Analysis Agent
# ---------------------------
class PreferenceAgent:
    def __init__(self, pipeline):
        self.pipeline = pipeline

    def generate_preferences(self, genres, language="English"):
        """
        Generate a detailed description of the user's movie ending preferences.
        This prompt instructs the model to use the user's preferred language and genres,
        including a brief summary of narrative structure, emotional tone, and the user's favorite genre.
        """
        genre_text = ", ".join(genres)
        prompt = (
            f"Using the user's preferred language ({language}) and genres ({genre_text}), "
            "describe the style of movie endings the user is most likely to enjoy. "
            "Your answer should be concise, detailing the narrative structure, emotional tone, and highlighting their favorite genre for the ending."
        )
        output = self.pipeline(prompt, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9)[0]['generated_text']
        # Remove the prompt portion if present
        generated_preferences = output[len(prompt):].strip()
        return generated_preferences

# ---------------------------
# Agent 3: Narrative Generation Agent
# ---------------------------
class NarrativeAgent:
    def __init__(self, pipeline):
        self.pipeline = pipeline

    def generate_narrative(self, preference_text, language="English"):
        """
        Generate a vivid, creative narrative paragraph describing an alternate movie ending.
        The narrative should reflect the preferences described in the input and be written in the specified language.
        """
        prompt = (
            f"Based on the following movie ending preferences: {preference_text}\n"
            f"Write a vivid and immersive narrative paragraph that describes an alternate movie ending. "
            f"The narrative should be in {language} and clearly reflect the user's taste, using detailed and engaging storytelling."
        )
        output = self.pipeline(prompt, max_new_tokens=200, do_sample=True, temperature=0.8, top_p=0.9)[0]['generated_text']
        generated_narrative = output[len(prompt):].strip()
        return generated_narrative

# ---------------------------
# Main function to tie the agents together
# ---------------------------
def main():
    # Example user data: watch history and comments
    watch_history = [
        "Inception",
        "The Matrix",
        "Interstellar",
        "Titanic",
        "The Notebook"
    ]

    comments = [
        "I love mind-bending sci-fi movies with deep plots.",
        "The emotional intensity of romance movies really moves me.",
        "I enjoy movies that keep me on the edge of my seat."
    ]

    # Step 1: Profile the user to get their preferred genres
    profiler = UserProfiler()
    user_genres = profiler.profile_user(watch_history, comments)
    print("User Preferred Genres:", user_genres)

    # Step 2: Generate movie ending preferences based on the user genres
    pref_agent = PreferenceAgent(text_gen_pipeline)
    preferences = pref_agent.generate_preferences(user_genres, language="English")
    print("\nGenerated Movie Ending Preferences:")
    print(preferences)

    # Step 3: Generate a narrative paragraph for a movie ending based on the preferences
    narrative_agent = NarrativeAgent(text_gen_pipeline)
    narrative = narrative_agent.generate_narrative(preferences, language="English")
    print("\nGenerated Movie Ending Narrative:")
    print(narrative)

if __name__ == "__main__":
    main()


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


User Preferred Genres: ['sci-fi', 'romance', 'drama', 'action']


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Generated Movie Ending Preferences:
Based on the user's preferences, they are most likely to enjoy movie endings that are:

**Narrative Structure:** Bittersweet, with a focus on character development and emotional payoff. The ending often features a mix of closure and new beginnings, leaving room for interpretation and emotional resonance.

**Emotional Tone:** Heartfelt, introspective, and sometimes melancholic. The ending typically evokes feelings of hope, acceptance, and a sense of growth, rather than a traditional happy ending.

**Favorite

Generated Movie Ending Narrative:
**

As the sun dipped below the horizon, casting a warm orange glow over the desolate landscape, Maya stood alone, her feet bare and her eyes closed. The wind whispered secrets in her ear, and the silence was a balm to her weary soul. The journey that had brought her to this moment was one of self-discovery, of shedding the skin of her past and emerging anew. The memories of her loved ones, of the laughter and t